In [6]:
import cv2
import numpy as np
import time
from IPython.display import clear_output
import math
import requests
import time

#TODO: свитч на кнопки, словарь с активностями
#TODO: отрефакторить это произведение искусства
#TODO: поменять клавиатурный интерфейс хождения
#TODO: словарь на действия и кнопки, а то совсем плохо
#TODO: добавить хэндлинг русских букв
#TODO: добавить флаг на работу в автономном режиме, без робота
#TODO: добавить кнопки на: руку вперед, "привет", разжать/сжать пальцы, танец

WIND_X = 200
WIND_Y = 200




cv2.namedWindow  ("remote_controller", cv2.WINDOW_AUTOSIZE)
cv2.resizeWindow ("remote_controller", (WIND_Y, WIND_X))

ip_prefix  = "http://"

#ip_num = "10.197.241.216"
# ip_num = "10.0.0.102"
ip_num = "172.20.10.14"
# ip_num = "192.168.43.65"
# ip_num = "192.168.1.29"

#ip_num = "192.168.43.42"
ip_postfix = ":"

ip = ip_prefix + ip_num + ip_postfix

port = "9562"

canvas = np.ones ((WIND_Y, WIND_X, 3), np.uint8) * 100

hello = [        #очень приятно
         "/?action=/stand&text=qwer",
        "/?action=/rest&text=qwer",
        "/?action=/stand&text=qwer"]

activity1 = ["/?action=/play_water&text=qwer",
             "/?action=/hands_sides&text=open_right",
             "/?action=/stand&text=qwer",
             "/?action=/play_dog_breathe&text=qwer",
             "/?action=/hands_sides&text=open_right",
             "/?action=/stand&text=qwer",
             "/?action=/play_dog_cry&text=qwer",
             "/?action=/hands_sides&text=open_right",
             "/?action=/stand&text=qwer",
             "/?action=/play_parrot_2&text=qwer"
            ] # повторяй за мной руки в бок

activity2 = ["/?action=/play_car&text=qwer",
             "/?action=/play_airplane_2&text=qwer",            
             "/?action=/hands_front&text=qwer",
             "/?action=/stand&text=qwer"
            ] # меня создали люди 

activity3 = ["/?action=/play_cat_eating&text=qwer",
            "/?action=/play_cat_meow&text=qwer",
             "/?action=/play_dog_bark&text=qwer",
             ] # как твои дела мои хорошо

activity4 = ["/?action=/play_dog_breathe&text=qwer",
             "/?action=/hands_front&text=qwer",
             "/?action=/stand&text=qwer",
             "/?action=/play_dog_cry&text=qwer",
             "/?action=/hands_front&text=qwer",
             "/?action=/stand&text=qwer",
             "/?action=/play_parrot_2&text=qwer",
            ] # руки вперед




activity5 = ["/?action=/play_dog_bark&text=qwer",
             "/?action=/hands_front&text=qwer"]




activity6 = ["/?action=/play_zhel_tuk_tok&text=qwer",
             "/?action=/play_zhel_zayka&text=qwer",
             "/?action=/play_zhel_logo_zha_zhu&text=qwer",
             "/?action=/play_zhel_logo_ku_gi&text=qwer",
             "/?action=/dance&text=qwer"]

ip1 = "192.168.1.249:9562"
ip2 = "192.168.1.29:9562"

activity7 = ["/?action=/right_hand_right&text=qwer",
             "192.168.1.29:9562/?action=/&text=qwer",
             "192.168.1.29:9562/?action=/right_hand_right&text=qwer",
             "192.168.1.29:9562/?action=/play_zhel_zayka&text=qwer",
             "192.168.1.29:9562/?action=/right_hand_right&text=qwer",
             "192.168.1.29:9562/?action=/play_zhel_zayka&text=qwer",]

queue  = []
queue_ = []

to_next_operation = False

time_ = 0

free_counter     = 0
max_free_counter = 10

curr_time = time.time ()
time_of_prev_press = 0.0

r = "66666666666666666666666"









dict_of_commands = {'z': '/?action=/stand&text=qwer', # встать
                   'w': '/?action=/dance&text=qwer', # танцевать
                   'e': '/?action=/hands_front&text=qwer', # руки вперед
                   'r': '/?action=/hands_sides&text=open_right', # руки вбок
                   't': '/?action=/hands&text=close_right', # закрыть правую руку
                   'f': '/?action=/play_water&text=qwer', # смотри как я умею
                   'x': '/?action=/sit&text=qwer', # сесть
                   'c': '/?action=/rest&text=qwer', # корточки
                   'v': '/?action=/play_dog_breathe&text=qwer', # повторяй за мной
                   'b': '/?action=/rot_50&text=qwer', # повернуться на 50 влево
#                     '/?action=/play_airplane_1&text=qwer', # привет я нао
                   'y': '/?action=/play_dog_cry&text=qwer', # попробуй еще
                   'g': '/?action=/play_parrot_2&text=qwer', # молодец
                   'h': '/?action=/hello', # привет
                   'm': '/?action=/walk_m30&text=qwer', # назад 30
                   'j': '/?action=/rot_20&text=qwer', # повернуться на 20 влево
                   'u': '/?action=/walk_50&text=qwer', # вперед 50
                   'i': '/?action=/walk_20&text=qwer', # вперед 20
                   'k': '/?action=/rot_m20&text=qwer', # 
                   'l': '/?action=/rot_m50&text=qwer', # 
                   'p': '/?action=/hands&text=open_right', # открыть правую руку
                    '1': '/?action=/M1',
                    '2': '/?action=/M2',
                    '3': '/?action=/M3',
                    '4': '/?action=/M4',
                    '5': '/?action=/M5',
                    '6': '/?action=/M6',
                   }




def list_and_dict_with_ord (dict_of_commands):
    list_of_keys = []
    dict_with_ord = {}
    for i in list(dict_of_commands.keys()):
        list_of_keys.append(ord(i))
        dict_with_ord.update({ord(i): dict_of_commands[i]})
    return list_of_keys, dict_with_ord

list_of_keys, dict_with_ord = list_and_dict_with_ord(dict_of_commands)
    
mode_without_queue = False
while (True):    
    cv2.waitKey (1)
    
    curr_time = time.time ()
    
    cv2.imshow ("remote_controller", canvas)
    
    time.sleep  (0.03)
    
    #handle keyboard events
    keyb = cv2.waitKey (1)
    
#     #action = 'free'
#     text = 'qwer'
#     #r = requests.get (ip + port + "/" + "?" + "action=/" + action + "&" + "text=" + text)
#     #print ("status ", int (str (r) [13:14]))
    
#     free = 0#int (str (r) [13:14]) #6 free, 7 not free; don't ask, don't tell
     
#     if (free == 6 and len (queue) != 0):
#         #clear screen
#         clear_output (wait=True)
        
#         print ("next: ", queue [0], len (queue_) - len (queue) + 1, "/", len (queue_))
        
#         if (to_next_operation == True):        
#             requests.get (ip + port + queue [0])
#             queue.remove (queue [0])
#             to_next_operation = False
    
#     action = 'free'
#     text = 'qwer'
    
    free_counter += 1
    
#     if free == 7 or len(queue) > 0:
#         if (free_counter == max_free_counter):
#             free_counter = 0
#             r = requests.get (ip + port + "/" + "?" + "action=/" + action + "&" + "text=" + text)
    
#     #print ("status ", int (str (r) [13:14]))
    
#     free = int (str (r) [13:14]) #6 free, 7 not free; don't ask, don't tell
    
#     free = 6
    
    if (len (queue) != 0 and to_next_operation == True):
        action = 'free'
        text = 'qwer'
        r = requests.get (ip + port + "/" + "?" + "action=/" + action + "&" + "text=" + text)
        free = int (str (r) [13:14]) #6 free, 7 not free; don't ask, don't tell
        if free == 6:
            time.sleep(0.1)
            requests.get (ip + port + queue [0])
            queue.remove (queue [0])
            if not mode_without_queue:
                to_next_operation = False
    
    
#     if (free == 6 and len (queue) != 0 and to_next_operation == True):
#         time.sleep(0.5)
#         requests.get (ip + port + queue [0])
#         queue.remove (queue [0])
#         if not mode_without_queue:
#             to_next_operation = False
    
    
    if (keyb != -1):
        upd = True
    
    if (keyb & 0xFF == ord ('q')):
        break
    
    elif (keyb & 0xFF == ord(' ')):
        mode_without_queue = not mode_without_queue
    
    elif (keyb & 0xFF == ord ('s')):
        queue [:] = []
        r = requests.get (ip + port + "/?action=/stop&text=qwer") # остановить действие

    elif (keyb & 0xFF == ord ('a')):
        queue  [:] = []
        queue_ [:] = []
    
    elif (keyb in list_of_keys):
#         print(keyb)
        queue.append(dict_with_ord[keyb])
    
#     elif (keyb & 0xFF == ord ('d')):
#         if (keyb & 0xFF == ord('z')):
#             print('123')
    
    elif (keyb & 0xFF == ord ('0')):
        #if (curr_time - time_of_prev_press > 0.4):
        #    time_of_prev_press = curr_time
        to_next_operation = True
        
        
    
    
#     elif (keyb & 0xFF == ord ('z')):
#         r = requests.get (ip + port + "/?action=/stand&text=qwer") # встать
    
#     elif (keyb & 0xFF == ord ('x')):
#         r = requests.get (ip + port + "/?action=/sit&text=qwer") # сесть 
    
#     elif (keyb & 0xFF == ord ('c')):
#         r = requests.get (ip + port + "/?action=/rest&text=qwer") # корточки

    

#     elif (keyb & 0xFF == ord ('w')):
#         r = requests.get (ip + port + "/?action=/dance&text=qwer") # танцевать

#     elif (keyb & 0xFF == ord ('e')):
#         r = requests.get (ip + port + "/?action=/hands_front&text=qwer") # руки вперед

#     elif (keyb & 0xFF == ord ('r')):
#         r = requests.get (ip + port + "/?action=/hands_sides&text=open_right") # руки вбок

    #elif (keyb & 0xFF == ord ('e')):
    #    r = requests.get (ip + port + "/?action=/right_hand_front&text=qwer")

    #elif (keyb & 0xFF == ord ('r')):
    #    r = requests.get (ip + port + "/?action=/hands&text=open_right")


#     elif (keyb & 0xFF == ord ('t')):
#         r = requests.get (ip + port + "/?action=/hands&text=close_right") # закрыть правую руку
    
#     elif (keyb & 0xFF == ord ('p')):
#         r = requests.get (ip + port + "/?action=/hands&text=open_right") # открыть правую руку
    
#     elif (keyb & 0xFF == ord ('i')):
#         r = requests.get (ip + port + "/?action=/walk_20&text=qwer") # вперед два-три шага
            
#     elif (keyb & 0xFF == ord ('u')):
#         r = requests.get (ip + port + "/?action=/walk_50&text=qwer") # вперед 5 шагов

#     elif (keyb & 0xFF == ord ('m')):
#         r = requests.get (ip + port + "/?action=/walk_m30&text=qwer") # назад 4 шага
        
#     elif (keyb & 0xFF == ord ('h')):
#         r = requests.get (ip + port + "/?action=/rot_50&text=qwer") # повернуться на 50 влево

#     elif (keyb & 0xFF == ord ('j')):
#         r = requests.get (ip + port + "/?action=/rot_20&text=qwer") # повернуться на 20 влево

#     elif (keyb & 0xFF == ord ('k')):
#         r = requests.get (ip + port + "/?action=/rot_m20&text=qwer") # 

#     elif (keyb & 0xFF == ord ('l')):
#         r = requests.get (ip + port + "/?action=/rot_m50&text=qwer") #
        
        
#     elif (keyb & 0xFF == ord ('b')):
#         r = requests.get (ip + port + "/?action=/play_airplane_1&text=qwer") # привет я нао
        
    
#     elif (keyb & 0xFF == ord ('v')):
#         r = requests.get (ip + port + "/?action=/play_dog_breathe&text=qwer") # повторяй за мной
#     elif (keyb & 0xFF == ord ('y')):
#         r = requests.get (ip + port + "/?action=/play_dog_cry&text=qwer") #попробуй еще
#     elif (keyb & 0xFF == ord ('g')):
#         r = requests.get (ip + port + "/?action=/play_parrot_2&text=qwer") # молодец
#     elif (keyb & 0xFF == ord ('f')):
#         r = requests.get (ip + port + "/?action=/play_water&text=qwer") # смотри как я умею
    
#  #play_train
    
#     elif (keyb & 0xFF == ord ('0')):
#         #if (curr_time - time_of_prev_press > 0.4):
#         #    time_of_prev_press = curr_time
#         to_next_operation = True
    
#     if (len (queue) == 0):
#         if (keyb & 0xFF == ord ('1')):
#             queue [:] = []
#             print (queue)
#             queue = activity1.copy ()

#         elif (keyb & 0xFF == ord ('2')):
#             queue  [:] = []
            
#             queue  = activity2.copy ()

#         elif (keyb & 0xFF == ord ('3')):
#             queue  [:] = []
            
#             queue  = activity3.copy ()

#         elif (keyb & 0xFF == ord ('4')):
#             queue  [:] = []
            
#             queue  = activity4.copy ()

#         elif (keyb & 0xFF == ord ('5')):
#             queue  [:] = []
#             queue_ [:] = []
            
#             queue  = activity5.copy ()

#         elif (keyb & 0xFF == ord ('6')):
#             queue  [:] = []
            
#             queue = activity6.copy ()

    curr_time = time.time ()

#     print(curr_time - time_)

    time_ = curr_time

cv2.waitKey           (1)
cv2.destroyAllWindows ()

ConnectionError: HTTPConnectionPool(host='172.20.10.14', port=9562): Max retries exceeded with url: /?action=/free&text=qwer (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000015FB5F60A88>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера'))

In [10]:
0.03417015075683594
0.03394055366516113
0.03439736366271973
0.03496837615966797
0.033951759338378906
0.03587913513183594
0.03479743003845215
0.03383922576904297
0.0348057746887207
0.0345609188079834
0.034661293029785156
0.03488564491271973

0.03477668762207031
0.034897804260253906
0.033599853515625
 0.060788631439208984
0.03452634811401367
0.034255027770996094
0.03497624397277832
0.03423333168029785
 0.1684873104095459
0.03411293029785156
0.03486442565917969
0.034300804138183594
0.03389859199523926
 0.15177249908447266
0.03513455390930176
0.034163475036621094
0.03474283218383789
0.03448987007141113
 0.1623826026916504
0.03422236442565918
0.03455615043640137
0.03445172309875488
0.03523588180541992
 0.1694035530090332
0.035109758377075195
0.03570556640625
0.034584760665893555
0.03447866439819336
 0.0478367805480957
0.03590822219848633
0.03436923027038574
0.0345001220703125
0.03471732139587402
 0.052178382873535156
0.03444504737854004
0.03551030158996582
0.03484392166137695

0.03488564491271973

In [1]:
    elif (keyb & 0xFF == ord ('i')):
        r = requests.get (ip + port + "/?action=/walk_20&text=qwer")
            
    elif (keyb & 0xFF == ord ('u')):
        r = requests.get (ip + port + "/?action=/walk_50&text=qwer")

    elif (keyb & 0xFF == ord ('m')):
        r = requests.get (ip + port + "/?action=/walk_m30&text=qwer")
        
    elif (keyb & 0xFF == ord ('h')):
        r = requests.get (ip + port + "/?action=/rot_50&text=qwer")

    elif (keyb & 0xFF == ord ('j')):
        r = requests.get (ip + port + "/?action=/rot_20&text=qwer")

    elif (keyb & 0xFF == ord ('k')):
        r = requests.get (ip + port + "/?action=/rot_m20&text=qwer")

    elif (keyb & 0xFF == ord ('l')):
        r = requests.get (ip + port + "/?action=/rot_m50&text=qwer")

    elif (keyb & 0xFF == ord ('1')):
        r = requests.get (ip + port + "/?action=/play_dog_cry&text=qwer")
 
    elif (keyb & 0xFF == ord ('a')):
        r = requests.get (ip + port + "/?action=/red&text=qwer")

    elif (keyb & 0xFF == ord ('s')):
        r = requests.get (ip + port + "/?action=/green&text=qwer")

    elif (keyb & 0xFF == ord ('d')):
        r = requests.get (ip + port + "/?action=/blue&text=qwer")

    elif (keyb & 0xFF == ord ('f')):
        r = requests.get (ip + port + "/?action=/right_hand_front&text=qwer")mrmr

    elif (keyb & 0xFF == ord ('g')):
        r = requests.get (ip + port + "/?action=/hands&text=open_right")

    elif (keyb & 0xFF == ord ('b')):
        r = requests.get (ip + port + "/?action=/hands&text=close_right")

    #elif (keyb & 0xFF == ord ('n')):
    #    r = requests.get (ip + port + "/?action=/lightblue&text=close_right")

    elif (keyb & 0xFF == ord ('v')):
        r = requests.get (ip + port + "/?action=/violet&text=close_right")

    elif (keyb & 0xFF == ord ('n')):
        r = requests.get (ip + port + "/?action=/set_right_hand_angle&text=close_right")



SyntaxError: invalid syntax (<ipython-input-1-efc073e9fe8e>, line 1)

In [3]:
import numpy as np
import cv2
import time
from IPython.display import clear_output

def get_available_cameras (upper_bound = 10, lower_bound = 0):
    available = []
    
    for i in range (lower_bound, upper_bound):
        cap = cv2.VideoCapture (i)
    
        if (cap.isOpened ()):
            available.append (i)
    
        cap.release ()
    
    return available

get_available_cameras ()

[0, 1]

In [1]:
import cv2
import numpy as np
import time
from IPython.display import clear_output
import math
import requests
import time

def get_middle_point(image):
    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(image)
    if (len (stats) > 1): 
        i_max = 1
        for i in range(2, len(stats)):
            if stats[i, cv2.CC_STAT_AREA] > stats[i_max, cv2.CC_STAT_AREA]:
                i_max = i
        x = stats [i_max, cv2.CC_STAT_LEFT] + (stats [i_max, cv2.CC_STAT_WIDTH]) / 2
        y = stats [i_max, cv2.CC_STAT_TOP] + (stats [i_max, cv2.CC_STAT_HEIGHT]) / 2
        return x,y
    return -1,-1

#TODO: свитч на кнопки, словарь с активностями
#TODO: отрефакторить это произведение искусства
#TODO: поменять клавиатурный интерфейс хождения
#TODO: словарь на действия и кнопки, а то совсем плохо
#TODO: добавить хэндлинг русских букв
#TODO: добавить флаг на работу в автономном режиме, без робота
#TODO: добавить кнопки на: руку вперед, "привет", разжать/сжать пальцы, танец

WIND_X = 200
WIND_Y = 200




cv2.namedWindow  ("remote_controller", cv2.WINDOW_AUTOSIZE)
cv2.resizeWindow ("remote_controller", (WIND_Y, WIND_X))

ip_prefix  = "http://"

#ip_num = "10.197.241.216"
# ip_num = "10.0.0.103"
# ip_num = "192.168.43.65"
ip_num = "192.168.1.29"

#ip_num = "192.168.43.42"
ip_postfix = ":"

ip = ip_prefix + ip_num + ip_postfix

port = "9562"

canvas = np.ones ((WIND_Y, WIND_X, 3), np.uint8) * 100

hello = ["/?action=/play_airplane_1&text=qwer",  #привет
         "/?action=/play_airplane_2&text=qwer",  #как тебя зовут
         "/?action=/right_hand_front&text=qwer", #
         "/?action=/play_car&text=qwer",         #очень приятно
         "/?action=/stand&text=qwer"]

activity1 = hello + \
            ["/?action=/play_cat_eating&text=qwer",
             "/?action=/play_cat_meow&text=qwer"]

activity2 = hello + \
            ["/?action=/play_dog_bark&text=qwer",
             "/?action=/play_dog_breathe&text=qwer"]

activity3 = hello + \
            ["/?action=/play_dog_cry&text=qwer",
             "/?action=/right_hand_right&text=qwer",
             "/?action=/play_parrot_1&text=qwer",
             "/?action=/stand&text=qwer"]

activity4 = hello + \
            ["/?action=/play_parrot_2&text=qwer",
             #"/?action=/shoulders_up&text=qwer",
             #"/?action=/stand&text=qwer",
             "/?action=/play_train&text=qwer",
             "/?action=/play_water&text=qwer"]

activity5 = hello + \
            ["/?action=/play_zhel_hands_up&text=qwer",
             "/?action=/play_zhel_top_hlop&text=qwer",
             "/?action=/shoulders_up&text=qwer",
             "/?action=/stand&text=qwer"]

activity6 = hello + \
            ["/?action=/play_zhel_tuk_tok&text=qwer",
             "/?action=/play_zhel_zayka&text=qwer",
             "/?action=/play_zhel_logo_zha_zhu&text=qwer",
             "/?action=/play_zhel_logo_ku_gi&text=qwer",
             "/?action=/dance&text=qwer"]
    
queue  = []
queue_ = []

to_next_operation = False

curr_time = time.time ()
time_of_prev_press = 0.0

cam1 = cv2.VideoCapture (1)

cv2.namedWindow ("frame", cv2.WINDOW_NORMAL)

cv2.resizeWindow ("frame", (960, 720))

while (True):
    ret1, frame1 = cam1.read ()
    
    cv2.waitKey (1)
    
    ker_sz = 19
    
    frame1_hsv = cv2.cvtColor(frame1, cv2.COLOR_BGR2HSV)   
    
    lower_red = np.array([0,120,70])
    upper_red = np.array([10,255,255])
    
    mask_red = cv2.inRange(frame1_hsv, lower_red, upper_red)
    
    lower_blue = np.array([120,120,70])
    upper_blue = np.array([140,255,255])
    
    mask_blue = cv2.inRange(frame1_hsv, lower_blue, upper_blue)
    
    
    clear_output (wait=True)
    
    x_blue, y_blue = get_middle_point(mask_blue)
    
    x_red, y_red = get_middle_point(mask_red)
    
#     frame = frame1 + 100
    
#     resized = cv2.resize (frame, (960, 720))
    
#     cv2.imshow ("frame1", resized)
#     cv2.imshow ("mask_red", mask_red)
#     cv2.imshow ("mask_blue", mask_blue)
    mask = np.concatenate((mask_blue, mask_red), axis=1)
    cv2.imshow('mask', mask)
    
    distance = abs(x_blue - x_red)
    
    if distance < 100:
        r = requests.get (ip + port + "/?action=/hands_front&text=qwer")

    elif distance >= 100:
        r = requests.get (ip + port + "/?action=/hands_sides&text=open_right")
    
    print(distance)
    time.sleep (0.01)
    
    if (cv2.waitKey (1) & 0xFF == ord('q')):
        break

cam1.release ()

cv2.destroyAllWindows()

ConnectionError: HTTPConnectionPool(host='192.168.1.29', port=9562): Max retries exceeded with url: /?action=/hands_sides&text=open_right (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001EB30BFA548>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера'))

In [2]:
import sys
from naoqi import ALProxy

PORT = 9559
robotIP = "192.168.1.29"
motionProxy = ALProxy("ALMotion", robotIP, PORT)
motionProxy.wakeUp()

# print motion state
print(motionProxy.getSummary())



ModuleNotFoundError: No module named 'naoqi'

In [ ]:
import paho.mqtt.client as mqtt
import json
import math
import almath
import time
import argparse
from naoqi import ALProxy

# Global variables
listAngles = []
shoulderLeft = []
elbowLeft = []
wristLeft = []
shoulderRight = []
elbowRight = []
wristRight = []
t = 0
RobotIP = raw_input("Geef robot IP: ")
RobotPort = int(raw_input("Geef robot Port (standaard 9559): "))
MQTTIP = raw_input("Geef mqtt IP: ")
MQTTTOPIC = raw_input("Geef mqtt topic: ")

def sendrobot(anglelist, robotIP="172.30.248.120", PORT=9559):
    try:
        try:
            motionProxy = ALProxy("ALMotion", robotIP, PORT) #creates proxy to call specific functions
        except Exception, e:
            print "Could not create proxy to AlMotion"
            print "Error was: ", e
        try:
            postureProxy = ALProxy("ALRobotPosture", robotIP, PORT) #creates proxy to call specific functions
        except Exception, e:
            print "Could not create proxy to ALRobotPosture"
            print "Error was: ", e

        global t # uses global variable t

        if (t == 0): # if it is the first time the robot is called upon
            motionProxy.setStiffnesses("Body", 0.0) # unstiffens the joints
            postureProxy.goToPosture("StandInit", 0.5) # gets the robot into his initial standing position

        names = ["RShoulderPitch", "RShoulderRoll", "RElbowRoll", "RElbowYaw",  "LShoulderPitch", "LShoulderRoll", "LElbowRoll", "LElbowYaw"]
        #list of joints that will get changed

        angleLists = [[(anglelist[len(anglelist) - 8]) * almath.TO_RAD], # all the coordinates are saved in one big list
                      [(anglelist[len(anglelist) - 7]) * almath.TO_RAD], # and in a specific order (see list of joints)
                      [(anglelist[len(anglelist) - 6]) * almath.TO_RAD], # this gets them out of that list and sent to the right joint
                      [(anglelist[len(anglelist) - 5]) * almath.TO_RAD],
                      [(anglelist[len(anglelist) - 4]) * almath.TO_RAD],
                      [(anglelist[len(anglelist) - 3]) * almath.TO_RAD],
                      [(anglelist[len(anglelist) - 2]) * almath.TO_RAD],
                      [(anglelist[len(anglelist) - 1]) * almath.TO_RAD]]
        timeLists = [[0.4], [0.4], [0.4], [0.4], [0.4], [0.4], [0.4], [0.4]] # sets the time the robot has to get to the joint location (when you give more than one coordinate for a joint, you have to give more than one timestamp for that same joint!)
        isAbsolute = True # kindoff is deprecated, but makes the joint positions absolute and not relative
        motionProxy.angleInterpolation(names, angleLists, timeLists, isAbsolute) #the function talks with the robot
        t += 1 # global t gets added by 1 so the joints dont get unstiffened again and the robot does not get put in its initial position
    except Exception: # checks for any and all errors
        pass # ignores every single one of them, except keyboardInterupt and SystemExit
    except (KeyboardInterrupt, SystemExit): # when the program gets terminated
        postureProxy.goToPosture("StandInit", 0.5) # set the robot in its initial position
        motionProxy.setStiffnesses("Body", 1.0) # stiffen the joints
        raise #actually quit

def angleRShoulderPitch(x2, y2, z2, x1, y1, z1): #calulates the Shoulderpitch value for the Right shoulder by using geometry
    if(y2<y1):
        angle = math.atan(abs(y2 - y1) / abs(z2 - z1)) 
        angle = math.degrees(angle)
        angle = -(angle)
        if(angle<-118):
            angle = -117
        return angle
    else:
        angle = math.atan((z2-z1)/(y2-y1))
        angle = math.degrees(angle)
        angle = 90-angle
        return angle

def angleRShoulderRoll(x2, y2, z2, x1, y1, z1): #calulates the ShoulderRoll value for the Right shoulder by using geometry
    if(z2<z1):
        test = z2
        anderetest = z1
        z2=anderetest
        z1=test
    if (z2 - z1 < 0.1):
        z2 = 1.0
        z1 = 0.8
    angle = math.atan((x2 - x1) / (z2 - z1))
    angle = math.degrees(angle)
    return angle

def angleLShoulderPitch(x2, y2, z2, x1, y1, z1): #calulates the Shoulderpitch value for the Left shoulder by using geometry
    if (y2 < y1):
        angle = math.atan(abs(y2 - y1) / abs(z2 - z1))
        angle = math.degrees(angle)
        angle = -(angle)
        if (angle < -118):
            angle = -117
        return angle
    else:
        angle = math.atan((z2 - z1) / (y2 - y1))
        angle = math.degrees(angle)
        angle = 90 - angle
        return angle

def angleLShouderRoll(x2, y2, z2, x1, y1, z1): #calulates the ShoulderRoll value for the Left shoulder by using geometry
    if (z2 < z1):
        test = z2
        anderetest = z1
        z2 = anderetest
        z1 = test
    if(z2-z1< 0.1):
        z2=1.0
        z1=0.8
    angle = math.atan((x2-x1)/(z2-z1))
    angle = math.degrees(angle)
    return angle

def angleRElbowYaw(x2, y2, z2, x1, y1, z1,shoulderpitch): #calulates the ElbowYaw value for the Right elbow by using geometry
    if(abs(y2-y1)<0.2 and abs(z2-z1) < 0.2 and (x1<x2) ):
        return 0
    elif(abs(x2-x1)<0.1 and abs(z2-z1)<0.1 and (y1>y2)):
        return 90
    elif(abs(x2-x1)<0.1 and abs(z2-z1)<0.1 and (shoulderpitch > 50)):
        return 90
    elif(abs(y2-y1)<0.1 and abs(z2-z1)<0.1 and (shoulderpitch < 50)):
        return 0
    elif(abs(x2-x1)<0.1 and abs(y2-y1)<0.1 and (shoulderpitch > 50)):
        return 90
    else:
        angle = math.atan((z2 - z1) / (y2 - y1))
        angle = math.degrees(angle)
        angle = - angle + (shoulderpitch)
        angle = - angle
        return angle


def angleRElbowRoll(x3, y3, z3, x2, y2, z2, x1, y1, z1): #calulates the ElbowRoll value for the Right elbow by using geometry
    a1=(x3-x2)**2+(y3-y2)**2 + (z3-z2)**2 
    lineA= a1 ** 0.5                        # calculates length of line between 2 3D coordinates
    b1=(x2-x1)**2+(y2-y1)**2 + (z2-z1)**2
    lineB= b1 ** 0.5                        # calculates length of line between 2 3D coordinates
    c1=(x1-x3)**2+(y1-y3)**2 + (z1-z3)**2
    lineC= c1 ** 0.5                        # calculates length of line between 2 3D coordinates

    cosB = (pow(lineA, 2) + pow(lineB,2) - pow(lineC,2))/(2*lineA*lineB)
    acosB = math.acos(cosB)
    angle = math.degrees(acosB)
    angle = 180 - angle
    return angle


def angleLElbowYaw(x2, y2, z2, x1, y1, z1, shoulderpitch): #calulates the ElbowYaw value for the Left elbow by using geometry
    if(abs(y2-y1)<0.2 and abs(z2-z1) < 0.2 and (x1>x2) ):
        return 0
    elif(abs(x2-x1)<0.1 and abs(z2-z1)<0.1 and (y1>y2)):
        return -90
    elif(abs(x2-x1)<0.1 and abs(z2-z1)<0.1 and (shoulderpitch > 50)):
        return -90
    elif(abs(y2-y1)<0.1 and abs(z2-z1)<0.1 and (shoulderpitch > 50)):
        return 0
    elif(abs(x2-x1)<0.1 and abs(y2-y1)<0.1 and (shoulderpitch > 50)):
        return -90
    else:
        angle = math.atan((z2 - z1) / (y2 - y1))
        angle = math.degrees(angle)
        angle = - angle + (shoulderpitch)
        angle = - angle
        return angle

def angleLElbowRoll(x3, y3, z3, x2, y2, z2, x1, y1, z1): #calulates the ElbowRoll value for the Left elbow by using geometry

    a1=(x3-x2)**2+(y3-y2)**2 + (z3-z2)**2
    lineA= a1 ** 0.5                        # calculates length of line between 2 3D coordinates
    b1=(x2-x1)**2+(y2-y1)**2 + (z2-z1)**2
    lineB= b1 ** 0.5                        # calculates length of line between 2 3D coordinates
    c1=(x1-x3)**2+(y1-y3)**2 + (z1-z3)**2
    lineC= c1 ** 0.5                        # calculates length of line between 2 3D coordinates

    cosB = (pow(lineA, 2) + pow(lineB,2) - pow(lineC,2))/(2*lineA*lineB)
    acosB = math.acos(cosB)
    angle = math.degrees(acosB)
    angle = -180+ angle
    return angle

def on_connect(client, userdata, flags, rc): # connects with mqtt and subscribes to /Sandro
    print("Connected with result code " + str(rc))
    client.subscribe(MQTTTOPIC)
    client.subscribe(MQTTTOPIC+str(2))

def on_message(client, userdata, msg): # Checks the mqtt message it receives and processes the json
    payload = json.loads(msg.payload.decode('utf-8'))
    if(msg.topic == "/Sandro"):
        for i in payload:
            if i['jointname'] == "ShoulderLeft":
                shoulderLeft = i['coordinates']
            if i['jointname'] == "ElbowLeft":
                elbowLeft = i['coordinates']
            if i['jointname'] == "WristLeft":
                wristLeft = i['coordinates']
            if i['jointname'] == "ShoulderRight":
                shoulderRight = i['coordinates']
            if i['jointname'] == "ElbowRight":
                elbowRight = i['coordinates']
            if i['jointname'] == "WristRight":
                wristRight = i['coordinates']

                listAngles.append(
                    angleRShoulderPitch(shoulderRight[0], shoulderRight[1], shoulderRight[2], elbowRight[0], elbowRight[1],
                                        elbowRight[2]))
                listAngles.append(
                    angleRShoulderRoll(shoulderRight[0], shoulderRight[1], shoulderRight[2], elbowRight[0], elbowRight[1],
                                    elbowRight[2]))
                listAngles.append(
                    angleRElbowRoll(shoulderRight[0], shoulderRight[1], shoulderRight[2], elbowRight[0], elbowRight[1],
                                    elbowRight[2], wristRight[0], wristRight[1], wristRight[2]))
                listAngles.append(
                    angleRElbowYaw(elbowRight[0], elbowRight[1], elbowRight[2], wristRight[0], wristRight[1],
                                wristRight[2], angleRShoulderPitch(shoulderRight[0], shoulderRight[1], shoulderRight[2], elbowRight[0], elbowRight[1],
                                        elbowRight[2])))
                listAngles.append(
                    angleLShoulderPitch(shoulderLeft[0], shoulderLeft[1], shoulderLeft[2], elbowLeft[0], elbowLeft[1],
                                        elbowLeft[2]))
                listAngles.append(
                    angleLShouderRoll(shoulderLeft[0], shoulderLeft[1], shoulderLeft[2], elbowLeft[0], elbowLeft[1],
                                    elbowLeft[2]))
                listAngles.append(
                    angleLElbowRoll(shoulderLeft[0], shoulderLeft[1], shoulderLeft[2], elbowLeft[0], elbowLeft[1],
                                    elbowLeft[2], wristLeft[0], wristLeft[1], wristLeft[2]))
                listAngles.append(
                    angleLElbowYaw(elbowLeft[0], elbowLeft[1], elbowLeft[2], wristLeft[0], wristLeft[1],
                                wristLeft[2], angleLShoulderPitch(shoulderLeft[0], shoulderLeft[1], shoulderLeft[2], elbowLeft[0], elbowLeft[1],
                                        elbowLeft[2])))
        sendrobot(listAngles, RobotIP, RobotPort) # takes userinput
    if(msg.topic == "/Sandro2"):
        x=1
        while x <= len(payload)-1:
            for key,value in payload.iteritems():
                numcoord = "coord" + str(x)
                if(key == numcoord ):
                    for i in value:
                        if i['jointname'] == "ShoulderLeft": # checks jointname in json
                            shoulderLeft = i['coordinates'] # puts the corresponding coordinates in the global list
                        if i['jointname'] == "ElbowLeft": # checks jointname in json
                            elbowLeft = i['coordinates'] # puts the corresponding coordinates in the global list
                        if i['jointname'] == "WristLeft": # checks jointname in json
                            wristLeft = i['coordinates'] # puts the corresponding coordinates in the global list
                        if i['jointname'] == "ShoulderRight": # checks jointname in json
                            shoulderRight = i['coordinates'] # puts the corresponding coordinates in the global list
                        if i['jointname'] == "ElbowRight": # checks jointname in json
                            elbowRight = i['coordinates'] # puts the corresponding coordinates in the global list
                        if i['jointname'] == "WristRight": # checks jointname in json
                            wristRight = i['coordinates'] # puts the corresponding coordinates in the global list

                    listAngles.append(angleRShoulderPitch(shoulderRight[0], shoulderRight[1], shoulderRight[2], elbowRight[0], elbowRight[1],elbowRight[2])) # calculates the angles via the Function with given coordinates and appends them to the masterlist
                    listAngles.append(angleRShoulderRoll(shoulderRight[0], shoulderRight[1], shoulderRight[2], elbowRight[0], elbowRight[1], elbowRight[2])) # calculates the angles via the Function with given coordinates and appends them to the masterlist
                    listAngles.append(angleRElbowRoll(shoulderRight[0], shoulderRight[1], shoulderRight[2], elbowRight[0], elbowRight[1], elbowRight[2], wristRight[0], wristRight[1], wristRight[2])) # calculates the angles via the Function with given coordinates and appends them to the masterlist
                    listAngles.append(angleRElbowYaw(elbowRight[0], elbowRight[1], elbowRight[2], wristRight[0], wristRight[1], wristRight[2], angleRShoulderPitch(shoulderRight[0], shoulderRight[1], shoulderRight[2], elbowRight[0], elbowRight[1], elbowRight[2]))) # calculates the angles via the Function with given coordinates and appends them to the masterlist
                    listAngles.append(angleLShoulderPitch(shoulderLeft[0], shoulderLeft[1], shoulderLeft[2], elbowLeft[0], elbowLeft[1], elbowLeft[2])) # calculates the angles via the Function with given coordinates and appends them to the masterlist
                    listAngles.append(angleLShouderRoll(shoulderLeft[0], shoulderLeft[1], shoulderLeft[2], elbowLeft[0], elbowLeft[1], elbowLeft[2])) # calculates the angles via the Function with given coordinates and appends them to the masterlist
                    listAngles.append(angleLElbowRoll(shoulderLeft[0], shoulderLeft[1], shoulderLeft[2], elbowLeft[0], elbowLeft[1], elbowLeft[2], wristLeft[0], wristLeft[1], wristLeft[2])) # calculates the angles via the Function with given coordinates and appends them to the masterlist
                    listAngles.append(angleLElbowYaw(elbowLeft[0], elbowLeft[1], elbowLeft[2], wristLeft[0], wristLeft[1], wristLeft[2], angleLShoulderPitch(shoulderLeft[0], shoulderLeft[1], shoulderLeft[2], elbowLeft[0], elbowLeft[1], elbowLeft[2]))) # calculates the angles via the Function with given coordinates and appends them to the masterlist
            sendrobot(listAngles, RobotIP, RobotPort) # Takes userinput


            x+=1



client = mqtt.Client() # mqtt stuff
client.on_connect = on_connect # mqtt stuff

client.on_message = on_message # mqtt stuff

client.connect(MQTTIP, 1883, 60) # takes userinput
client.loop_forever() # listen forever